# Divide-and-Conquer: Strassen's Algorithm

Let $A = (a_{ij})_{n\times n}$ and $B = (b_{ij})_{n\times n}$ be two square matrices of the same dimensions. Then multiplying $A$ and $B$ is defined by
$$C = A \times B = (c_{ij})_{n\times n},$$
where $c_{ij} = \sum_{k=1}^n a_{ik} \cdot b_{kj}$.
This can be implemented in Python as following:

In [3]:
import numpy as np
def multiply(A, B):
    # first check if the inputs are square matrices of the same dimensions
    m = A.shape[0]
    if A.shape[1] != m:
        print(A, "\n must be a square matrix.")
        return
    n = B.shape[0]
    if m != n:
        print("\n dimenstion mismatched.")
        return
    if B.shape[1] != n:
        print(B, "\n must be a square matrix.")
        return
    # Now compute the product
    C = np.zeros((n,n), dtype=np.int64)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k] * B[k,j]
    return C

In [4]:
A = np.matrix([[1, 2 ,3],
               [3, 1, 1],
               [4, 8 ,9]])

B = np.matrix([[4, 3, 1], 
               [7, 8, 2], 
               [9, 7, 3]])
multiply(A,B)

array([[ 45,  40,  14],
       [ 28,  24,   8],
       [153, 139,  47]], dtype=int64)

# Time Complexity

The time complexity of the matrix multiplication in the straightforward implementation is
$n \cdot n \cdot (n+n-1) = \Theta(n^3)$.

Question: Can we do better?

Let's explore if divide and conquer could help. First we would want to divide the original problem into subproblems. To this end, let's first assume that $n$ is a power of 2 so that it is divisible by 2 recursively. Then $A$ and $B$ can each be divided into four matrics as 
$$
A = \left(
\begin{array}{cc}
a & b \\
c & d
\end{array}
\right),
~~ 
B = \left(
\begin{array}{cc}
e & f \\
g & h
\end{array}
\right).
$$
If we multiple them directly as before, then we have the following recurrence relation:
$$
C = \left(
\begin{array}{cc}
a\cdot e + b\cdot g & a\cdot f + c\cdot h \\
c\cdot e + d\cdot g & c\cdot f + d\cdot h
\end{array}
\right).
$$
Let $T(n)$ be the number of operations for multiplying $A$ and $B$. Then
$$T(n) = 8T\left(\frac{n}{2}\right) + c\left(\frac{n}{2}\right)^2, ~T(1) = 1,$$
for some constant $c > 0$.
This gives rise to the following
$$
T(n) 
> 8T\left(\frac{n}{2}\right) 
> 8\left(8T\left(\frac{n}{2^2}\right)\right) 
= 8^2T\left(\frac{n}{2^2}\right) 
> 8^2\left(8T\left(\frac{n}{2^3}\right)\right) 
= 8^3T\left(\frac{n}{2^3}\right) 
> \cdots 
> 8^kT\left(\frac{n}{2^k}\right). 
$$
When $n/2^k = 1$, namely, when $k = \log n$, the recurrence stops.
Hence, $T(n) > 8^{\log n}T(1)  = n^3$.
This implies that the naive divid and conquer doesn't help. 

Thankfully, Strassen discovered relations that reduce 8 multiplications to 7 as shown in the following figure,
where the combine part doesn't envolve multiplications. 

<!--![image.png](Strassen.png)-->
<div>
   <img src="Strassen.png" width="450">
</div>

This is a big deal because under this new divide and conquer stragety,
we have
$T(n) = 7T(n/2) + c(n/2)^2$ form some constant $c>1$ and $T(1) = 1$. 
Now let's compute $T(n)$.
\begin{eqnarray*}
T(n) 
&=& 7T(n/2) + c(n/2)^2 \\
&=& 7(7T(n/2^2) + c(n/2^2)^2) + cn^2/4 \\
&=& 7^2T(n/2^2) + 7/2^4 cn^2 + 1/2^2cn^2 \\
&=& \cdots \\
&=& 7^kT(n/2^k) + cn^2/4\sum_{i=0}^{k-1} 7^i/2^{2i} \\
&=& 7^kT(n/2^k) + \Theta(n^2)
\end{eqnarray*}
When $k = \log n$, the recurrence stops and we have $7^kT(n/2^k) = 7^{\log n}T(1) = 7^{\log n}$.
Note that $\sum_{i=0}^{k-1} (7/4)^i = ((7/4)^k - 1)/(7/4-1) = c'7^{\log n}/n^2$. Thus,
$T(n) = \Theta(7^{\log n})$. Note that $\log 7 < 2.81 < 3$.

When $n$ is not a power of 2, we pad $A$ and $B$. In particular, we may just need to pad one additional column of 0's on the right and one additional row of 0's on the bottom when $n$ is an odd number since $n+1$ is even and so can be divided by 2.
Note that $N < 2n$, and so the asymptotic complexity remains the same. That is,
$$
A' = \left(
\begin{array}{cc}
A & 0 \\
0 & 0
\end{array}
\right)
\text{ and } 
B' = \left(
\begin{array}{cc}
B & 0 \\
0 & 0
\end{array}
\right).
$$
To compute $A\cdot B$, it's equivalent to computing 
$$A'\cdot B' = 
\left(
\begin{array}{cc}
A & 0 \\
0 & 0
\end{array}
\right)
\cdot
\left(
\begin{array}{cc}
B & 0 \\
0 & 0
\end{array}
\right)
= 
\left(
\begin{array}{cc}
AB & 0 \\
0 & 0
\end{array}
\right).
$$
We use Strassen's algorithm to compute it in $\Theta(n^{\log 7})$ time. After that, we remove
1 column from the right and row from the bottom if during the recursion, the corresponding number is padded.
This produces the product of $A$ and $B$.

In [5]:
def strassen_algorithm(A, B):
    # first check if the inputs are square matrices of the same dimensions
    p = A.shape[0]
    if A.shape[1] != p:
        print(A, "\n must be a square matrix.")
        return
    q = B.shape[0]
    if p != q:
        print("\n dimenstion mismatched.")
        return
    if B.shape[1] != q:
        print(B, "\n must be a square matrix.")
        return
    
    # divide and conquer halting condition
    if A.size == 1 or B.size == 1:
        return A * B
    
    n = A.shape[0]
    # Let's see if we need to pad
    if n % 2 == 1: # n is odd; pad
        A = np.pad(A, (0, 1), mode='constant') # add an additional column on the right and a row on the bottom of 0's
        B = np.pad(B, (0, 1), mode='constant')
    
    m = int(np.ceil(n / 2)) # divide
    a = A[:m, :m]
    b = A[:m, m:]
    c = A[m:, :m]
    d = A[m:, m:]
    e = B[:m, :m]
    f = B[:m, m:]
    g = B[m:, :m]
    h = B[m:, m:]
    p1 = strassen_algorithm(a, f - h) # subproblems
    p2 = strassen_algorithm(a + b, h)
    p3 = strassen_algorithm(c + d, e)
    p4 = strassen_algorithm(d, g - e)
    p5 = strassen_algorithm(a + d, e + h)
    p6 = strassen_algorithm(b - d, g + h)
    p7 = strassen_algorithm(a - c, e + f)
    C = np.zeros((2 * m, 2 * m), dtype=np.int64)
    C[:m, :m] = p5 + p4 - p2 + p6
    C[:m, m:] = p1 + p2
    C[m:, :m] = p3 + p4
    C[m:, m:] = p1 + p5 - p3 - p7

    return C[:n, :n]

In [6]:
A = np.matrix([[1, 2 ,3, 3],
               [3, 1, 1, 5],
               [4, 8 ,9, 10],
               [2, 3, 8, 1]])

B = np.matrix([[4, 3, 1, 4], 
               [7, 8, 2, 7], 
               [9, 7, 3, 3],
               [2, 3, 1, 4]])
strassen_algorithm(A,B)

array([[ 51,  49,  17,  39],
       [ 38,  39,  13,  42],
       [173, 169,  57, 139],
       [103,  89,  33,  57]], dtype=int64)

In [7]:
import random

m = 9
n = 2 ** m
L_A = [[]] * n
L_B = [[]] * n
for i in range(n):
    L_A[i] = [random.randrange(1, 100) for i in range(n)] # generate 20k random numbers
for i in range(n):
    L_B[i] = [random.randrange(1, 100) for i in range(n)] # generate 20k random numbers


In [8]:
M_A = np.matrix(L_A)
M_B = np.matrix(L_B)

In [9]:
M_A.shape[1] == M_A.shape[0]

True

In [10]:
M_B.shape[1] == M_B.shape[0]

True

In [11]:
strassen_algorithm(M_A, M_B)

KeyboardInterrupt: 

In [12]:
multiply(M_A, M_B)

KeyboardInterrupt: 